In [1]:
# required imports
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
#getting the club URLs
pl_home_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
pl_home_response = requests.get(pl_home_url, verify = False)
pl_home_soup = BeautifulSoup(pl_home_response.content, 'html.parser')
overall_teams_table = pl_home_soup.find('table', id = 'results2022-202391_overall')
all_urls = [link.get('href') for link in overall_teams_table.findAll("a")]
team_urls = [url for url in all_urls if 'en/squads' in url]

In [4]:
for team_url in team_urls:
    response = requests.get('https://fbref.com' + team_url, verify = False)
    team_soup = BeautifulSoup(response.content, 'html.parser')
    team_table = team_soup.find('table', id = 'stats_standard_9')
    team_name = team_url.split('/')[-1][:-6].replace('-', ' ')

    for row in team_table.find_all('tr'):
            positions = row.find_all("td", {"data-stat" : "position"})
            for position in positions:
                if (position.text == 'FW'):
                    player_name = row.find("th", {"data-stat" : "player"}).text

                    player_url = 'https://fbref.com' + row.find('a', attrs={'href': re.compile("/en/players")}).get('href')
                    all_matches_url = '/'.join(player_url.split('/')[:-1]) + '/matchlogs/2022-2023/c9/summary/' + player_url.split('/')[-1] + '-Match-Logs'
                    pl_matches_url = all_matches_url.split('summary')[0] + 'c9/summary' + all_matches_url.split('summary')[1]

                    filename = team_name + ' - ' + player_name + '.csv'
        
                    pl_match_log_response = requests.get(pl_matches_url, verify = False)
                    pl_match_log_soup = BeautifulSoup(pl_match_log_response.content, 'html.parser')
                    pl_match_log_table = pl_match_log_soup.find_all('table', id = 'matchlogs_2022-2023_9')

                    if pl_match_log_table:
                        df = pd.read_html(str(pl_match_log_table))[0]
                        df.columns = df.columns.droplevel()
                        df = df.drop(columns=['Match Report'])
                        df = df.dropna()
                        df.to_csv('PlayerData/' + filename, index=False)
                    else: 
                        print(r"Couldn't fetch data for " + player_name + ' from ' + team_name)
            
